__Purpose:__ Introduce Federated Learning, specifically by implementing FedAveraging on our dataset and moving on to more advanced methods.  Start by modifying the Simulations code, worry about (a)synchronicity later.
<br>
1. The dec matrix is the weights to pass back an forth (I think), although it comes out of SmoothBatch first
1. We are assuming we can test on the second half (updates 10-19ish) since learning should be complete by then!
1. Scipy.optimize.minimize() runs many iters to fully minimize its cost function.  You can change it to run as many iters as you'd like, although AFAIK you won't know how many it takes to converge.  But this is still a good set up for FL.
1. Hmm minimize() is doing BFGS rn and not SGD... not sure if that matters really.  Could probably implement SGD on my own or find it.  BFGS is 2nd order but we don't have a lot of parameters, I don't think.  Plus we can (already have?) solved analytically for the Hessian.

In [1]:
import pandas as pd
import os
import numpy as np
#from numpy.matlib import repmat
#from matplotlib import pyplot as plt
#from scipy.signal import detrend, firwin, freqz, lfilter
#from sklearn.model_selection import train_test_split, ShuffleSplit
from scipy.optimize import minimize, least_squares
import copy
from itertools import permutations

In [4]:
from experiment_params import *
from simulations import *
import time
# Do the below if you're in the pytch environment
#import pickle5 as pickle
import pickle

## Load Our Data In

In [5]:
'''
t0 = time.time()
emg_data_df1 = pd.read_csv("Data\emg_full_data1.csv")
emg_data_df2 = pd.read_csv("Data\emg_full_data2.csv")
emg_data_df = pd.concat((emg_data_df1, emg_data_df2))
try:
    emg_data_df.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("NO UNNAMED COLUMN DETECTED!")
t1 = time.time()
total = t1-t0  
print(total)
print(emg_data_df.shape)
emg_data_df.head()
'''
# Just use the emg data directly from the pickle file for now
;

''

In [6]:
'''
t0 = time.time()
#envelope_df50 = pd.read_csv("Data\envelope_df50.csv")
envelope_df100 = pd.read_csv("Data\envelope_df100.csv")
#envelope_df150 = pd.read_csv("Data\envelope_df150.csv")
#envelope_df200 = pd.read_csv("Data\envelope_df200.csv")
#envelope_df250 = pd.read_csv("Data\envelope_df250.csv")
#envelope_df300 = pd.read_csv("Data\envelope_df300.csv")
#raw_envs = [envelope_df50, envelope_df100, envelope_df150, envelope_df200, envelope_df250, envelope_df300]
#all_envs = [env.drop('Unnamed: 0', axis=1) for env in raw_envs]
try:
    envelope_df100.drop('Unnamed: 0', axis=1, inplace=True)
except:
    print("NO UNNAMED COLUMN DETECTED!")
t1 = time.time()
total = t1-t0  
print(total)
print(envelope_df100.shape)
envelope_df100.head()
'''
# Just use the emg data directly from the pickle file for now
;

''

In [7]:
t0 = time.time()

with open('Data\continuous_full_data_block1.pickle', 'rb') as handle:
    #refs_block1, poss_block1, dec_vels_block1, int_vel_block1, emgs_block1, Ws_block1, Hs_block1, alphas_block1, pDs_block1, times_block1, conditions_block1 = pickle.load(handle)
    refs_block1, _, _, _, emgs_block1, Ws_block1, _, _, _, _, _ = pickle.load(handle)

#with open('Data\continuous_full_data_block2.pickle', 'rb') as handle:
    #refs_block2, poss_block2, dec_vels_block2, int_vel_block2, emgs_block2, Ws_block2, Hs_block2, alphas_block2, pDs_block2, times_block2, conditions_block2 = pickle.load(handle)
    #refs_block2, _, _, _, emgs_block2, Ws_block2, _, _, _, _, _ = pickle.load(handle)

t1 = time.time()
total = t1-t0  
print(total)

12.349754095077515


In [8]:
# 8 conditions, 20770 data points (only 19 unique sets!), xy, channels
Ws_block1[keys[0]].shape

(8, 20770, 2, 64)

In [9]:
update_ix

array([    0,  1200,  2402,  3604,  4806,  6008,  7210,  8412,  9614,
       10816, 12018, 13220, 14422, 15624, 16826, 18028, 19230, 20432,
       20769])

In [10]:
dec_cond0_user1_update0 = Ws_block1[keys[0]][0,0,:,:]
dec_cond0_user1_update1 = Ws_block1[keys[0]][0,update_ix[1],:,:]
dec_cond0_user1_update2 = Ws_block1[keys[0]][0,update_ix[2],:,:]

print(f"Shape of decoder: {dec_cond0_user1_update0.shape}")
print()
print(f"Total difference between dec0 and dec1: {(dec_cond0_user1_update0 - dec_cond0_user1_update1).sum()}")
print("E.g., as previously shown, the first two decs are the same")
print()
print(f"Total difference between dec0 and dec2: {(dec_cond0_user1_update0 - dec_cond0_user1_update2).sum()}")

Shape of decoder: (2, 64)

Total difference between dec0 and dec1: 0.0
E.g., as previously shown, the first two decs are the same

Total difference between dec0 and dec2: 3.1981579823181594


In [12]:
#emg_cond0_user1_update0 = emg_data_df.iloc[:64,:].shape

# (Condition, datapoints, channels)
print(emgs_block1[keys[0]][:,:,:].shape)

# Condition 0 of subject 1 ("0")
print(emgs_block1[keys[0]][0,:,:].shape)

(8, 20770, 64)
(20770, 64)


## Run One Iteration On Above Data and Check Decoders Are the Same
1. Modifying Simulations Code

In [18]:
# Just 1 person
filtered_signals = emgs_block1[keys[0]][0,:,:]
# Read in the reference positions from the pickle file
cued_target_position = refs_block1[keys[0]][0,:,:]

print(filtered_signals.shape)
print(cued_target_position.shape)

(20770, 64)
(20770, 2)


In [13]:
# Previously created random decoder, but we are trying to rerun
#D_0 = np.random.rand(2,64)
D_0 = dec_cond0_user1_update0

#learning_batch = 8
learning_batch = update_ix[1]  # I think this is supposed to be the number of datapoints per update?... why was it only 8 before then? Still don't know where they were getting 60 data points from
#num_dps_per_update = update_ix[1]  #1200

In [14]:
alpha = .95 # higher alpha means more old decoder (slower update)
alphaF=1e-1
alphaD = 1e-1

D = []
D_constant = []
D_bounded = []
D_constant_bounded = []
D.append(D_0)
D_constant.append(D_0)
D_bounded.append(D_0)
D_constant_bounded.append(D_0)
accuracy = []
accuracy_constant = []
accuracy_bounded = []
accuracy_constant_bounded = []

Assuming I don't need to take all these different funcs into account... should probably ask

In [15]:
# Original code for running simulations...
# for ix in range(10000):
    #accuracy_constant_,D_constant,p_constrained_constant = simulation_constant_intent(D_constant,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)
    #accuracy_constant.extend(accuracy_constant_)
    #accuracy_,D,p_constrained = simulation(D,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)    
    #accuracy.extend(accuracy_)
    #accuracy_bounded_,D_bounded,p_bounded = simulation_bounded_pos(D_bounded,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)  
    #accuracy_bounded.extend(accuracy_bounded_)
    #accuracy_constant_bounded_,D_constant_bounded,p_constant_bounded = simulation_constant_intent_bounded(D_constant_bounded,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)
    #accuracy_constant_bounded.extend(accuracy_constant_bounded_)
    
# Modified code for running simulations...
# Why loop at all right now...
#for ix in range(10):
    #accuracy_,D,p_constrained = simulation(D,learning_batch,alpha,alphaF=alphaF,alphaD=alphaD)    


In [16]:
# Added 2 new parameters
#def simulation(D,learning_batch,alpha,alphaF=1e-2,alphaD=1e-2,display_info=False,num_iters=False):
#D  # Already defined
#learning_batch  # Already defined
#alpha  # Already defined
#alphaF=1e-2  #defined as something else earlier...
#alphaD=1e-2  #defined as something else earlier...
display_info=True
num_iters=False

In [17]:
#cond_number = 0 .... I don't need this because I already passed in the 0th condition to filtered signals... idk if I should keep it that way... I guess let's get it to run first.......
# What was the original shape and meaning of filtered_signals? It must have been (datapoints, ?, ?), one of those is channels idk about the other one

In [51]:
# Added 2 new parameters
#def simulation(D,learning_batch,alpha,alphaF=1e-2,alphaD=1e-2,display_info=False,num_iters=False):
p_classify = []
accuracy_temp = []

#num_updates = int(np.floor((filtered_signals.shape[0]-1)/learning_batch)) # how many times can we update decoder based on learning batch    
num_updates = 19  # This is 19 for us

# Nah idk what this is doing so I'm gonna remove it
# RANDOMIZE DATASET
#randomized_integers = np.random.permutation(range(0,cued_target_position.shape[0]))
#filtered_signals_randomized = filtered_signals[randomized_integers]
#cued_target_position_randomized = cued_target_position[randomized_integers]

# batches the trials into each of the update batch
for ix in range(num_updates):
    #s = np.hstack([x for x in filtered_signals[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:,:]])# stack s (64 x (60 timepoints x learning batch size))
    #p_intended = np.hstack([np.tile(x[:,np.newaxis],60) for x in cued_target_position[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    #v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)
    
    # What operation do I need to do to get this to be 64x1200?
    #s = np.hstack([x for x in np.transpose(filtered_signals[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:])])# stack s (64 x (60 timepoints x learning batch size))
    s = np.transpose(filtered_signals[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:])# stack s (64 x (60 timepoints x learning batch size))
    p_intended = np.transpose(cued_target_position[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]) # stack p_intended (2 x 60 timepoints x learning batch size)
    v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

    # CLASSIFY CURRENT DECODER ACCURACY
    #v_actual = D[-1]@s
    #for trial in range(learning_batch):
    #    # The 60s are messing things up...
    #    # Actually the 60s don't really affect things since they get summed out anyways, its just a window size more or less
    #    v_trial = v_actual[:,int(trial*60):int((trial+1)*60)] # velocities for each trials (2,60)
    #    p_final = np.sum(v_trial,axis=1)[:,np.newaxis] # final position after integration (2,)
    #    # What is this supposed to be? Target_positions was global, p_final is (2,) so I guess just a subset of target_positions?
    #    # They didn't do that I don't think tho lol
    #    p_classify.append(classify(p_final, p_intended)) 

    # UPDATE DECODER
    u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
    q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials

    # emg_windows against intended_targets (trial specific cued target)
    F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
    V = copy.deepcopy(q)

    # Ask why we have this (D0) and also D_0...
    # initial decoder estimate for gradient descent
    D0 = np.random.rand(2,64)

    # set alphas
    H = np.zeros((2,2))
    # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
    if num_iters is False:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info})
    else:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info, 'maxiter':num_iters})

    # reshape to decoder parameters
    W_hat = np.reshape(out.x,(2, 64))

    # DO SMOOTHBATCH
    W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
    D.append(W_new)

    # COMPUTE CLASSIFICATION ACURACY 
    # No randomized... maybe it's just cued_target_positions?
    #p_target = (cued_target_position[randomized_integers])[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:] # obtain target
    #p_target = cued_target_position
    #accuracy_temp.append(classification_accuracy(p_target,p_classify[-learning_batch:]))

p_classify = np.asarray(p_classify)
#return accuracy_temp,D,p_constrained

C:\Users\kdmen\AppData\Local\Temp\ipykernel_27568\4248677151.py:54: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info})


Optimization terminated successfully.
         Current function value: 21816.557630
         Iterations: 148
         Function evaluations: 199
         Gradient evaluations: 199
Optimization terminated successfully.
         Current function value: 21570.103379
         Iterations: 149
         Function evaluations: 193
         Gradient evaluations: 193
Optimization terminated successfully.
         Current function value: 26808.097431
         Iterations: 148
         Function evaluations: 189
         Gradient evaluations: 189
Optimization terminated successfully.
         Current function value: 29048.277741
         Iterations: 149
         Function evaluations: 203
         Gradient evaluations: 203
Optimization terminated successfully.
         Current function value: 24568.018656
         Iterations: 150
         Function evaluations: 188
         Gradient evaluations: 188
Optimization terminated successfully.
         Current function value: 20350.854089
         Iterations: 

In [53]:
len(D)

21

In [47]:
v_trial.shape

(2, 60)

In [49]:
p_final.shape

(2, 1)

In [39]:
76800/64

1200.0

In [40]:
(76800-72000)/1200

4.0

In [41]:
s.shape

(64, 1200)

In [42]:
D[-1].shape

(2, 64)

In [43]:
p_intended.shape

(2, 1200)

In [44]:
p_final.shape

(2, 1)

In [45]:
p_constrained.shape

(2, 1200)

In [ ]:
assert(1==0)

Don't remember what the stikt here is

In [37]:
p_classify = []
accuracy_temp = []

#num_updates = int(np.floor((filtered_signals.shape[0]-1)/learning_batch)) # how many times can we update decoder based on learning batch    
num_updates = 19  # Let this equal the number of actual updates

#############################################################################################
# RANDOMIZE DATASET
# Idk what's going on here
randomized_integers = np.random.permutation(range(0,cued_target_position.shape[0]))
filtered_signals_randomized = filtered_signals  # filtered_signals[randomized_integers]
cued_target_position_randomized = cued_target_position  # cued_target_position[randomized_integers]
#############################################################################################

# batches the trials into each of the update batch
for ix in range(num_updates):
    #s = np.hstack([x for x in filtered_signals_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:,:]])# stack s (64 x (60 timepoints x learning batch size))
    # stack s (64 x (60 timepoints x learning batch size))
    #s = filtered_signals_randomized[:, int(ix*learning_batch+1):int((ix+1)*learning_batch+1)]
    # Idk why they had all these +1s...
    s = np.transpose(filtered_signals_randomized[:, int(ix*learning_batch):int((ix+1)*learning_batch+1)])
    #p_intended = np.hstack([np.tile(x[:,np.newaxis],60) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    p_intended = np.hstack([np.tile(x[:,np.newaxis],learning_batch) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    # Band-aid solution
    p_intended = p_intended[:, :20770]
    v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

    # CLASSIFY CURRENT DECODER ACCURACY
    v_actual = D[-1]@s
    for trial in range(learning_batch):
        v_trial = v_actual[:,int(trial*num_dps_per_update):int((trial+1)*num_dps_per_update)] # velocities for each trials (2,60)
        p_final = np.sum(v_trial,axis=1)[:,np.newaxis] # final position after integration (2,)
        p_classify.append(classify(p_final, cued_target_position))

    # UPDATE DECODER
    u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
    q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials

    # emg_windows against intended_targets (trial specific cued target)
    F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
    V = copy.deepcopy(q)

    # initial decoder estimate for gradient descent
    D0 = np.random.rand(2,64)
    # Why is this different from D_0?

    # set alphas
    H = np.zeros((2,2))
    # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
    if num_iters is False:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info})
    else:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info, 'maxiter':num_iters})

    # reshape to decoder parameters
    W_hat = np.reshape(out.x,(2, 64))

    # DO SMOOTHBATCH
    W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
    D.append(W_new)

    # COMPUTE CLASSIFICATION ACURACY 
    # This is definitely wrong, not sure what it should be changed to...
    p_target = (cued_target_position[randomized_integers])[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:] # obtain target
    accuracy_temp.append(classification_accuracy(p_target,p_classify[-learning_batch:]))

p_classify = np.asarray(p_classify)
#return accuracy_temp,D,p_constrained

Optimization terminated successfully.
         Current function value: 21935.005158
         Iterations: 140
         Function evaluations: 188
         Gradient evaluations: 188


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 0 is different from 64)

In [36]:
s.shape

(1201, 64)

In [32]:
ix

1

In [31]:
len(D)

2

In [17]:
D[-1].shape

(2, 64)

In [30]:
s.shape

(0, 20770)

In [28]:
filtered_signals.shape

(20770, 64)

In [29]:
filtered_signals_randomized.shape

(20770, 64)

In [21]:
np.transpose(filtered_signals_randomized[:, int(ix*learning_batch):int((ix+1)*learning_batch+1)]).shape

(0, 20770)

In [23]:
int(ix*learning_batch)

1200

In [24]:
int((ix+1)*learning_batch+1)

2401

In [27]:
filtered_signals_randomized[:, 1200:2401].shape

(20770, 0)

In [33]:
filtered_signals.shape

(20770, 64)

In [61]:
s.shape[1]

1200

This one also breaks, usually it runs through once (ix gets to 1) but then breaks the second time around

In [62]:
p_classify = []
accuracy_temp = []

#num_updates = int(np.floor((filtered_signals.shape[0]-1)/learning_batch)) # how many times can we update decoder based on learning batch    
num_updates = 19  # Let this equal the number of actual updates

#############################################################################################
# RANDOMIZE DATASET
# Idk what's going on here
randomized_integers = np.random.permutation(range(0,cued_target_position.shape[0]))
filtered_signals_randomized = np.transpose(filtered_signals)  # filtered_signals[randomized_integers]
print(filtered_signals_randomized.shape)
cued_target_position_randomized = cued_target_position  # cued_target_position[randomized_integers]
print(cued_target_position_randomized.shape)
#############################################################################################

# batches the trials into each of the update batch
for ix in range(num_updates):
    #s = np.hstack([x for x in filtered_signals_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:,:]])# stack s (64 x (60 timepoints x learning batch size))
    # stack s (64 x (60 timepoints x learning batch size))
    #s = filtered_signals_randomized[:, int(ix*learning_batch+1):int((ix+1)*learning_batch+1)]
    # Idk why they had all these +1s...
    s = filtered_signals_randomized[:, int(ix*learning_batch):int((ix+1)*learning_batch)]
    #p_intended = np.hstack([np.tile(x[:,np.newaxis],60) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    p_intended = np.hstack([np.tile(x[:,np.newaxis],learning_batch) for x in cued_target_position_randomized[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:]]) # stack p_intended (2 x 60 timepoints x learning batch size)
    # Band-aid solution
    p_intended = p_intended[:, :s.shape[1]]
    v_intended,p_constrained = output_new_decoder(s,D[-1],p_intended)

    # CLASSIFY CURRENT DECODER ACCURACY
    v_actual = D[-1]@s
    for trial in range(learning_batch):
        v_trial = v_actual[:,int(trial*num_dps_per_update):int((trial+1)*num_dps_per_update)] # velocities for each trials (2,60)
        p_final = np.sum(v_trial,axis=1)[:,np.newaxis] # final position after integration (2,)
        p_classify.append(classify(p_final, cued_target_position))

    # UPDATE DECODER
    u = copy.deepcopy(s) # u is the person's signal s (64 CHANNELS X TIMEPOINTS)
    q = copy.deepcopy(v_intended) # use cued positions as velocity vectors for updating decoder should be 2 x num_trials

    # emg_windows against intended_targets (trial specific cued target)
    F = copy.deepcopy(u[:,:-1]) # note: truncate F for estimate_decoder
    V = copy.deepcopy(q)

    # initial decoder estimate for gradient descent
    D0 = np.random.rand(2,64)
    # Why is this different from D_0?

    # set alphas
    H = np.zeros((2,2))
    # use scipy minimize for gradient descent and provide pre-computed analytical gradient for speed
    if num_iters is False:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info})
    else:
        out = minimize(lambda D: cost_l2(F,D,H,V), D0, method='BFGS', jac = lambda D: gradient_cost_l2(F,D,H,V), options={'disp': display_info, 'maxiter':num_iters})

    # reshape to decoder parameters
    W_hat = np.reshape(out.x,(2, 64))

    # DO SMOOTHBATCH
    W_new = alpha*D[-1] + ((1 - alpha) * W_hat)
    D.append(W_new)

    # COMPUTE CLASSIFICATION ACURACY 
    # This is definitely wrong, not sure what it should be changed to...
    p_target = cued_target_position[-learning_batch:, :]  # (cued_target_position[randomized_integers])[int(ix*learning_batch+1):int((ix+1)*learning_batch+1),:] # obtain target
    accuracy_temp.append(classification_accuracy(p_target,p_classify[-learning_batch:]))

p_classify = np.asarray(p_classify)
#return accuracy_temp,D,p_constrained

(64, 20770)
(20770, 2)
Optimization terminated successfully.
         Current function value: 1106.912511
         Iterations: 148
         Function evaluations: 199
         Gradient evaluations: 199
Optimization terminated successfully.
         Current function value: 4708.949398
         Iterations: 149
         Function evaluations: 204
         Gradient evaluations: 204
Optimization terminated successfully.
         Current function value: 1213.727482
         Iterations: 149
         Function evaluations: 191
         Gradient evaluations: 191
Optimization terminated successfully.
         Current function value: 1619.567090
         Iterations: 150
         Function evaluations: 201
         Gradient evaluations: 201
Optimization terminated successfully.
         Current function value: 752.830458
         Iterations: 151
         Function evaluations: 195
         Gradient evaluations: 195
Optimization terminated successfully.
         Current function value: 1088.797473
     

ValueError: need at least one array to concatenate

In [58]:
s.shape

(64, 1200)

In [59]:
p_intended.shape

(2, 370)

In [60]:
D[-1].shape

(2, 64)

In [ ]:
print(D)